# Vẽ biểu đồ và so sánh độ trễ theo từng hop, PDR theo số lượng kênh

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

In [2]:
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext('local[*]')

In [3]:
import plotly
from plotly.graph_objs import Scatter, Layout, Bar, Figure, Layout

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

init_notebook_mode(connected=True)

# Đọc và tính toán các thông số về độ trễ từ tập các file
### Xử lý dữ liệu
#### Tính toán các thông số về độ lệch chuẩn, tính trung bình, ...

In [4]:
# filenames = [
#             'orchestra_log_2_channel.csv', 
#             'orchestra_log_3_channel.csv', 
#             'orchestra_log_4_channel.csv', 
#             'orchestra_log_5_channel.csv'
# #            ]
#             ,
# # filenames = [
#             'rlm_log_2_channel.csv', 
#             'rlm_log_3_channel.csv', 
#             'rlm_log_4_channel_diff.csv', 
#             'rlm_log_5_channel.csv'
#            ]
filenames = [
            'rlm-1-channels.csv',
            'rlm-2-channels.csv',
            'rlm-3-channels.csv',
            'rlm-4-channels.csv',
            'rlm-5-channels.csv'
           ]

means1hop = []
stdevs1hop = []
means2hop = []
stdevs2hop = []

means = []
stdevs = []

pdrs = []

time2hop1s = []
time2hop2s = []
latency2hop1s = []
latency2hop2s = []

for filename in filenames:
    tf = sc.textFile(filename)
    rdd = tf.map(lambda x: (x.split(";")[0], x)).groupByKey().mapValues(list)
    
    # calc PDR
    recv = rdd.filter(lambda x: len(x[1]) > 1).count()
    sent = rdd.filter(lambda x: x[1][0].split(";")[2] != "1").count()
    pdrs.append(recv / float(sent))
    
    # calc Latency
    hopping1 = rdd.filter(lambda x: len(x[1]) == 2)
    hopping2 = rdd.filter(lambda x: len(x[1]) == 3)
    
    # calc lantecy per packet
    hop1 = hopping1.map(lambda x: (x[1][0].split(";")[2] + '-' + x[1][0].split(";")[1], (int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1]))))    
    hop2 = hopping2.map(lambda x: (x[1][0].split(";")[2] + '-' + x[1][0].split(";")[1], (int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1]))))    
    
    time2hop1 = hop1.sortByKey().map(lambda x: x[0]).collect()
    time2hop2 = hop2.sortByKey().map(lambda x: x[0]).collect()
    latency2hop1 = hop1.sortByKey().map(lambda x: x[1]).collect()       
    latency2hop2 = hop2.sortByKey().map(lambda x: x[1]).collect()

    time2hop1s.append(time2hop1)
    time2hop2s.append(time2hop2)
    latency2hop1s.append(latency2hop1)
    latency2hop2s.append(latency2hop2)
    
    # calc LATENCY mean, standard deviation
    latency1 = hopping1.map(lambda x: int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
    latency2 = hopping2.map(lambda x: int(x[1][2].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
    
    means.append(np.mean(latency1 + latency2))
    stdevs.append(np.std(latency1 + latency2))
    
    means1hop.append(np.mean(latency1))
    means2hop.append(np.mean(latency2))
    stdevs1hop.append(np.std(latency1))
    stdevs2hop.append(np.std(latency2))
    
print 'pdrs    : ', pdrs
print 'means   : ', means
print 'stdevs  : ', stdevs
print 'means1  : ', means1hop
print 'stdevs1 : ', stdevs1hop
print 'means2  : ', means2hop
print 'stdevs2 : ', stdevs2hop

pdrs    :  [0.8235294117647058, 0.8878504672897196, 0.7478991596638656, 0.7478991596638656, 0.7317073170731707]
means   :  [957.96938775510205, 1060.9894736842105, 950.02247191011236, 750.89887640449433, 677.16666666666663]
stdevs  :  [958.08283752120644, 1044.9955275123314, 1031.8845090146722, 686.86046936824073, 430.55414551740438]
means1  :  [798.84810126582283, 815.21568627450984, 794.60810810810813, 630.28378378378375, 630.01666666666665]
stdevs1 :  [775.96926874760868, 1018.8727395321674, 863.29857378651218, 483.83226712364615, 450.15628736646073]
means2  :  [1619.578947368421, 1345.8636363636363, 1716.7333333333333, 1345.9333333333334, 771.4666666666667]
stdevs2 :  [1299.2148769156288, 1001.6680994246341, 1390.6790411721734, 1103.8631235599619, 370.82410325591775]


# Vẽ đồ thị

## Biểu đồ so sánh PDR

In [5]:
trace = Bar(
    x = filenames,
    y = pdrs,
    name='PDR'
)

data = [trace]
layout = Layout(
    title = 'Biểu đồ so sánh PDR',
    barmode = 'group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/PDR')

## Biểu đồ so sánh Latency

In [6]:
trace = Bar(
    x = filenames,
    y = means,
    name='Latency',
    error_y=dict(
        type = 'data',
        array = stdevs,
        visible = True
    )
)
data = [trace]
layout = Layout(
    title = 'Biểu đồ so sánh Latency',
    barmode = 'group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/Latency')

## Biểu đồ so sánh độ trễ theo từng hop

In [7]:
trace1 = Bar(
    x = filenames,
    y = means1hop,
    name='1 hopping',
    error_y=dict(
        type = 'data',
        array = stdevs1hop,
        visible = True
    )
)
trace2 = Bar(
    x = filenames,
    y = means2hop,
    name='2 hopping',
    error_y=dict(
        type = 'data',
        array = stdevs2hop,
        visible = True
    )
)

data = [trace1, trace2]
layout = Layout(
    title = 'Biểu đồ so sánh độ trễ theo từng hop',
    barmode = 'group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/Latency-per-hop')

## Biểu đồ so sánh độ trễ theo từng gói của 1 hop và 2 hop

In [23]:
data = []
for time2hop, latency2hop, filename in itertools.izip(time2hop1s, latency2hop1s, filenames):
    trace1 = Scatter(
        x = time2hop,
        y = latency2hop,
        mode = 'lines+markers',
        name = "1 Hop " + filename
    )
    data.append(trace1)
iplot(data, filename='jupyter/Latency-per-packet-1-hop')

data = []
for time2hop, latency2hop, filename in itertools.izip(time2hop2s, latency2hop2s, filenames):
    trace2 = Scatter(
        x = time2hop,
        y = latency2hop,
        mode = 'lines+markers',
        name = "2 Hop " + filename
    )
    data.append(trace2)

iplot(data, filename='jupyter/Latency-per-packet-2-hop')